In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import torch 
import tensorflow as tf
from sklearn.metrics import accuracy_score
data=pd.read_csv('IRIS.csv')
species=dict(zip(list(data['species'].unique()),([0,1,2])))
data['species'].replace(species,inplace=True)
y=data.pop('species')
y_dummy=pd.get_dummies(y)
y_dummy
X=data.values


2022-10-02 22:19:08.864584: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-10-02 22:19:15.241979: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-10-02 22:19:15.242026: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-10-02 22:19:15.752538: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-10-02 22:19:25.235533: W tensorflow/stream_executor/platform/de

In [2]:
X_train=tf.constant(X,dtype=tf.double)
y_dummy=tf.constant(y_dummy)

X_train.dtype

2022-10-02 22:19:33.138180: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-10-02 22:19:33.153908: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-10-02 22:19:33.153976: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (jabed-HP-Notebook): /proc/driver/nvidia/version does not exist
2022-10-02 22:19:33.194058: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


tf.float64

In [3]:
class Linear:
    
    def __init__(self,input_dim,units=3):
        #super(Linear,self).__init__()
        #self.weight=tf.Variable(np.random.randn(input_dim,units),trainable=True)
        self.weight=tf.Variable(np.random.randn(shape=(input_dim,units)),dtype=tf.float64)
        #self.bias=tf.Variable(np.zeros(units),trainable=True)
        self.bias=tf.Variable(np.zeros(units),dtype=tf.float64)
        
    def __call__(self,xb):
        self.x=tf.Variable(xb,dtype=tf.float64)
        #return tf.matmul(xb,self.weight)+self.bias 
        return (xb@self.weight)+self.bias

    def backward(self,dl_da):
        try:
                
            # dl/da comes from loss class 
            # shape of dl-da is= nx1 
            # shape of x is =nxd
            # output must be as weight shape =dx1
            # X.T shape is dxn 
            # out shape is of dl-dw = dxn * nx1=dx1
            # weights shape=dx1   dl_da shape is nx1
            #dl_da -(nx1) * transpose of weight (1xd) =nxd    dl_dx for mlp 
            self.dl_dw=self.x.T@dl_da
            #print(dl_da.dtypes)
            self.dl_db=dl_da.sum(axis=0)
            self.dx=dl_da@ self.weight.T # nxd
            return self.dx
        except:
            self.dl_dw=tf.transpose(self.x)@dl_da  
            #print('dl_da',dl_da.dtype) 
            self.dl_db=tf.reduce_sum(dl_da)
            self.dx=dl_da@(tf.transpose(self.weight))
            return self.dx 
    def update(self,lr=0.001):


        print(self.weight)
        #print('dl_dw',self.dl_dw.dtype)
        self.weight=self.weight+lr*self.dl_dw
        self.bias=self.bias+lr*self.dl_db
        #print(self.weight)

        

In [4]:
class softmax:
    def __call__(self,Z):
        self.z=Z
        z=np.exp(Z-np.max(Z))
        self.out=z/z.sum(axis=0,keepdims=True)
        return self.out
    def backward(self,dl):
        self.Z=tf.Variable(self.z)
        with tf.GradientTape() as tape:
            s=tf.nn.softmax(self.Z)
        soft_grad=tape.gradient(s,self.Z)
        return soft_grad*dl     


In [5]:
class crossentropy:
    def __call__(self,y_true,y_pred):
        
        self.y_true=y_true
        self.y_pred=tf.Variable(y_pred)
        self.loss=tf.reduce_mean(tf.losses.categorical_crossentropy(y_true,y_pred))
        return self.loss
    def backward(self):
        with tf.GradientTape() as tape:

            losses=tf.reduce_mean(tf.losses.categorical_crossentropy(self.y_true,self.y_pred))
        dl_da=tape.gradient(losses,self.y_pred)
        return dl_da    
        



In [6]:

for epoch in range(100):

    
    Z=linear(X_train)
    soft=softmax()
    y_pred=soft(Z)
    los=crossentropy()
    los(y_dummy,y_pred)
    dl=los.backward()
    ds=soft.backward(dl)
    linear_grad=linear.backward(ds)
    linear.update()

    

   
  




NameError: name 'linear' is not defined

In [ ]:
'''y_pred=tf.Variable(y_pred)
with tf.GradientTape() as tape:
    losses=tf.reduce_sum(loss(y_dummy,y_pred))
dl=tape.gradient(losses,y_pred)    

'''

In [ ]:
linear.weight.dtype
linear_grad.dtype